<a href="https://colab.research.google.com/github/chewzzz1014/fyp/blob/master/ner/src/train_ner_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train NER Models

In [ ]:
# mount drive

In [ ]:
# import CONLL2003 annotated dataset

In [ ]:
# split into train and test

## Spacy NER

In [2]:
# create dir to place spacy ner data
!mkdir spacy_ner_data

# convert CONLL2003 annotation data into spacy data
!python -m spacy convert "train.txt" spacy_ner_data -c ner
!python -m spacy convert "test.txt" spacy_ner_data -c ner
!python -m spacy convert "valid.txt" spacy_ner_data -c ner

sample_data


In [ ]:
# create base_config.cfg and paste the config generated from spacy widget
# update train and test file path
!touch base_config.cfg

In [ ]:
# generate config.cfg from base_config.cfg
!python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
# train model using hyperparameters set in config.cfg
# trained model in output/ dir
!python -m spacy train config.cfg --output ./output

In [ ]:
# download trained model